In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train.csv', 'sample_submission.csv']


In [50]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import pandas as pd
import numpy as np
import random
random.seed(7)
import time
import pickle
from catboost import CatBoostRegressor, Pool
#data scaling
from sklearn.preprocessing import StandardScaler
#hyperparameter optimization
from sklearn.model_selection import GridSearchCV
#support vector machine model
from sklearn.svm import NuSVR, SVR
#kernel ridge model
from sklearn.kernel_ridge import KernelRidge
#data visualization
import matplotlib.pyplot as plt

In [ ]:
%%time
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
df=train

In [ ]:
ttf = df['time_to_failure'].values
index_start = np.nonzero(np.diff(ttf) > 0)[0] + 1
index_start = np.insert(index_start, 0, 0)
dict_df={}
for i in range(len(index_start)):
    if i<(len(index_start)-1):
        df_tmp=df[index_start[i]:index_start[i+1]]
    else:
        df_tmp=df[index_start[i]:]       
    dict_df["df"+str(i)]=df_tmp

In [ ]:
def randomChoice(l):
    return random.randint(0, l - 1)

In [ ]:
def randomTrainingExample(df_dict):
    k = len(df_dict)
    num=randomChoice(k)
    #print(num)
    len_df=len(df_dict['df'+str(num)])
    idx_start=random.randint(0,len_df-149999)
    idx_end=idx_start+150000
    #print('Start Index:',idx_start)
    #print('End Index:',idx_end)
    df_tmp=df_dict['df'+str(num)]
    sample_x=df_tmp.iloc[idx_start:idx_end]['acoustic_data']
    sample_y=df_tmp.iloc[idx_start:idx_end]['time_to_failure']  
    sample_x=np.array(sample_x)
    sample_y=np.array(sample_y)    
    uid =str(num)+'-'+str(idx_start)+'-'+str(idx_end)
    return uid,sample_x,sample_y

In [ ]:
uid,xtrain,ytrain=randomTrainingExample(dict_df)


In [ ]:
dict_final_x={}
dict_final_y={}

In [ ]:
n_examples=5000
now=time.time()
while(n_examples):
    while(uid in dict_final_x.keys()):
        uid,xtrain,ytrain=randomTrainingExample(dict_df)
    xtrain=np.array(xtrain)
    ytrain=np.array(ytrain)
    xtrain= xtrain.astype(np.float32)
    ytrain= ytrain.astype(np.float32)
    dict_final_x[uid]=xtrain
    dict_final_y[uid]=ytrain
    n_examples-=1
then=time.time()
print("Time Taken:",then-now)

In [ ]:

num_epochs = 200
batch_size = 10
learning_rate = 1e-3

In [ ]:
dataloader = DataLoader(list(dict_final_x.keys()), batch_size=batch_size, shuffle=True)



In [ ]:
def get_x(uid_list):
        x=dict_final_x[uid_list]
        return x
        
def get_y(uid_list):
        y=dict_final_y[uid_list]
        return y

In [ ]:
from scipy.stats import skew
from scipy.stats import  kurtosis

In [ ]:
def gen_features(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(kurtosis(X))
    strain.append(skew(X))
    strain.append(np.quantile(X,0.01))
    strain.append(np.quantile(X,0.05))
    strain.append(np.quantile(X,0.95))
    strain.append(np.quantile(X,0.99))
    strain.append(np.abs(X).max())
    strain.append(np.abs(X).mean())
    strain.append(np.abs(X).std())
    return pd.Series(strain)

In [ ]:
num_epochs=170
print(dataloader)

In [ ]:
X_train=pd.DataFrame()
Y_train=pd.Series()
for data in dataloader:
#         print(data)
        for i in data:
            x_train = get_x(i)
#             print(x_train)
#             print(i)
#         print('Xtrain:')
#         print(x_train.shape)
            y_train=get_y(i)
            ch = gen_features(x_train)
            X_train = X_train.append(ch, ignore_index=True)
            Y_train = Y_train.append(pd.Series(y_train[-1]))
#         print("sdsd")
print(X_train.shape)
print(Y_train.shape)

In [ ]:
print()

In [ ]:

X_train.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
train_pool = Pool(X_train, Y_train)

In [ ]:
m = CatBoostRegressor(iterations=10000, loss_function='MAE', boosting_type='Ordered')
m.fit(X_train_scaled, Y_train)

In [ ]:
m.best_score_


In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame()

In [ ]:
for seg_id in submission.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    ch = gen_features(seg['acoustic_data'])
    X_test = X_test.append(ch, ignore_index=True)


In [ ]:
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [ ]:
Y_test = m.predict(X_test_scaled)

In [ ]:
submission['time_to_failure'] =Y_test
submission.to_csv('submission1.csv')

In [ ]:
submission